### Input: ROOT files with the THnSparses of the same-event and mixed-event correlations, ROOT file with response matrices, and the configuration files used to generate those correlations and response matrices

This notebook is just for plotting purposes. It uses the GammaJetCorrelation class, which converts the various histograms to numpy arrays, does various subtractions, and saves the histograms as numpy arrays. The notebook then takes these arrays and generates and saves the plots.

This is designed for a single iteration (i.e. a single configuration file), so comparing different iterations together will require modification.

### Workflow:
1. Read config files
2. Pull histograms and response matrices from ROOT file(s) and save as ROOT objects
3. Make gamma-jet correlation objects, which consist of the following steps:
    1. Subtract mixed-event from same-event for both signal-region and background-region. Save these histograms as numpy arrays (for plotting)
    2. Subtract background-region from signal-region for both same-event and mixed-event. Save these histograms as numpy arrays (for plotting)
    3. Subtract background-region from signal-region. Save this histogram as a numpy array (for plotting)
    4. Convert this fully-subtracted correlation to a TH1 for unfolding
    5. Unfold with response matrix
    6. Convert unfolded correlation to numpy array (for plotting)
4. Make and save plots

In [ ]:
workingdir=%pwd
%cd -q ../..

from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import os
import ROOT
import root_numpy as rnp
import shutil
import yaml

from gj_correlation import GammaJetCorrelation, getAllCorr, AxisNum
from plotstyle import *
from unfolder import Unfolder
from utils import *

ROOT.TH1.AddDirectory(False)
ROOT.TH2.AddDirectory(False)

%cd $workingdir
plotdir = 'plots'
if not os.path.exists(plotdir):
    os.mkdir(plotdir)

In [ ]:
# load all YAML files
with open('config/globalconfig.yaml') as configfile:
    configgj = yaml.safe_load(configfile)
    
with open('config/systemconfig.yaml') as configfile:
    configsys = yaml.safe_load(configfile)

with open('config/runconfig.yaml') as configfile:
    configrun = yaml.safe_load(configfile)

configall = {}
configall.update(configgj)
configall.update(configsys)
configall.update(configrun)

centranges = [tuple(centrange) for centrange in configall['centralityranges']]
photonptranges = [tuple(ptrange) for ptrange in configall['clusterptranges']]
observables = [obs['name'] for obs in configall['observables']]
observableInfo = {obs['name']: obs for obs in configall['observables']}

for observable in observables:
    obsdir = '{0}/{1}'.format(plotdir, observable)
    if not os.path.exists(obsdir):
        os.mkdir(obsdir)

# don't know why this doesn't get set when importing plotstyle, so do it again here
mpl.rc('figure', figsize=(8, 8))

In [ ]:
# print configuration for future reference
print(configall)

In [ ]:
# make any *minor* changes to the configuration here
# (e.g. jetpt cut, deltaphi cut, number of bins -- things that do not affect the creation of the THnSparses)
# this should only be done for exploring these variations
# any real changes should be made in the configuration file(s)
def setJetPtRange(ptmin, ptmax):
    observableInfo['deltaphi']['cuts']['jetpt'] = {'min': ptmin, 'max': ptmax, 'text': '{0} < $p_\mathrm{{T}}^\mathrm{{jet}}$ < {1} GeV/$c$'.format(ptmin, ptmax)}
    
def setDeltaPhiMin(num, den):
    # num, den: the numerator and denominator of the fraction of pi to be used as the minimum deltaphi pt cut
    observableInfo['jetpt']['cuts']['deltaphi'] = {'min': np.pi * num / den, 'max': np.pi, 'text': '{0}$\pi$/{1} < $\Delta\varphi$ < $\pi$'.format(num, den)}
    observableInfo['ptratio']['cuts']['deltaphi'] = {'min': np.pi * num / den, 'max': np.pi, 'text': '{0}$\pi$/{1} < $\Delta\varphi$ < $\pi$'.format(num, den)}

def setPtRatioRange(ptratiomin, ptratiomax):
    observableInfo['deltaphi']['cuts']['ptratio'] = {'min': ptratiomin, 'max': ptratiomax, 'text': '{0} < $p_\mathrm{{T}}^\mathrm{{jet}}$/$p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1}'.format(ptratiomin, ptratiomax)}

def useNbins(observable, nbins):
    if 120 % nbins == 0:
        observableInfo[observable]['nbins'] = nbins
    else:
        print('Cannot use {0} bins as it is not a multiple of 120'.format(nbins))

In [ ]:
# get correlation objects
rootfileSE = configall['filelists']['correlations']['sameevent']
rootfileME = configall['filelists']['correlations']['mixedevent']
rootfileMEskim5090 = rootfileME.replace('.root', '_skimcent5090.root')
rootfileRM = configall['filelists']['correlations']['responsematrix']

allCorr = getAllCorr(centranges, photonptranges, observableInfo, rootfileSE, rootfileME, rootfileRM)
allCorrskim5090 = getAllCorr([(50, 90)], photonptranges, observableInfo, rootfileSE, rootfileME, rootfileRM)
allCorr[(50, 90)] = allCorrskim5090[(50, 90)]

allCorr['longname'] = configall['longname']
allCorr['shortname'] = configall['shortname']

## Correlation plots

In [ ]:
# plot all raw correlations on same plot
for observable in observables:
    for centrange in centranges:
        fig = plt.figure(figsize=(len(photonptranges) * 8, 8))
        for (icol, photonptrange) in enumerate(photonptranges):
            gjCorr = allCorr[centrange][photonptrange][observable]

            plt.subplot(1, len(photonptranges), icol + 1)
            gjCorr.plotCorr('sesr', fmt='C0o', fillstyle='none', label='SESR')
            gjCorr.plotCorr('sebr', fmt='C0o-', fillstyle='none', alpha=0.4, label='SEBR')
            gjCorr.plotCorr('mesr', fmt='C1s', fillstyle='none', label='MESR')
            gjCorr.plotCorr('mebr', fmt='C1s-', fillstyle='none', alpha=0.4, label='MEBR')

            if icol == len(photonptranges) - 1:
                plt.legend(loc=observableInfo[observable]['legendLoc'])

            if observableInfo[observable]['isLog']:
                plt.yscale('log')

            plt.xlabel(observableInfo[observable]['xlabel'])

            if icol == 0:
                plt.ylabel(observableInfo[observable]['ylabel'])
                infotext = '{0}, {1}-{2}%'.format(allCorr['longname'], centrange[0], centrange[1])
                plt.annotate(infotext, **observableInfo[observable]['systemtextplacement'])

            infotext = '{0} < $p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1} GeV/$c$'.format(photonptrange[0], photonptrange[1])
            for cut in observableInfo[observable]['cuts']:
                infotext = infotext + '\n' + observableInfo[observable]['cuts'][cut]['text']
            plt.annotate(infotext, **observableInfo[observable]['pttextplacement'])

        fig.savefig('{0}/{1}/sesr-sebr-mesr-mebr-{2}-{3}-{4}.png'.format(plotdir, observable, allCorr['shortname'], centrange[0], centrange[1]))

In [ ]:
# make BR-subtracted SE and ME plots
for observable in observables:
    for centrange in centranges:
        fig = plt.figure(figsize=(len(photonptranges) * 8, 8))
        for (icol, photonptrange) in enumerate(photonptranges):
            gjCorr = allCorr[centrange][photonptrange][observable]

            plt.subplot(1, len(photonptranges), icol + 1)
            gjCorr.plotCorr('se', fmt='C0o', label='SE')
            gjCorr.plotCorr('me', fmt='C1s', label='ME')

            if icol == len(photonptranges) - 1:
                plt.legend(loc=observableInfo[observable]['legendLoc'])

            if observableInfo[observable]['isLog']:
                plt.yscale('log')

            plt.xlabel(observableInfo[observable]['xlabel'])

            if icol == 0:
                plt.ylabel(observableInfo[observable]['ylabel'])
                infotext = '{0}, {1}-{2}%'.format(allCorr['longname'], centrange[0], centrange[1])
                plt.annotate(infotext, **observableInfo[observable]['systemtextplacement'])

            infotext = '{0} < $p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1} GeV/$c$'.format(photonptrange[0], photonptrange[1])
            for cut in observableInfo[observable]['cuts']:
                infotext = infotext + '\n' + observableInfo[observable]['cuts'][cut]['text']
            plt.annotate(infotext, **observableInfo[observable]['pttextplacement'])

        fig.savefig('{0}/{1}/se-me-{2}-{3}-{4}.png'.format(plotdir, observable, allCorr['shortname'], centrange[0], centrange[1]))

In [ ]:
# make ME-subtracted SR and BR plots
for observable in observables:
    for centrange in centranges:
        fig = plt.figure(figsize=(len(photonptranges) * 8, 8))
        for (icol, photonptrange) in enumerate(photonptranges):
            gjCorr = allCorr[centrange][photonptrange][observable]

            plt.subplot(1, len(photonptranges), icol + 1)
            gjCorr.plotCorr('sr', fmt='C0o', label='SR')
            gjCorr.plotCorr('br', fmt='C1s', label='BR')

            if icol == len(photonptranges) - 1:
                plt.legend(loc=observableInfo[observable]['legendLoc'])

            if observableInfo[observable]['isLog']:
                plt.yscale('log')

            plt.xlabel(observableInfo[observable]['xlabel'])

            if icol == 0:
                plt.ylabel(observableInfo[observable]['ylabel'])
                infotext = '{0}, {1}-{2}%'.format(allCorr['longname'], centrange[0], centrange[1])
                plt.annotate(infotext, **observableInfo[observable]['systemtextplacement'])

            infotext = '{0} < $p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1} GeV/$c$'.format(photonptrange[0], photonptrange[1])
            for cut in observableInfo[observable]['cuts']:
                infotext = infotext + '\n' + observableInfo[observable]['cuts'][cut]['text']
            plt.annotate(infotext, **observableInfo[observable]['pttextplacement'])

        fig.savefig('{0}/{1}/sr-br-{2}-{3}-{4}.png'.format(plotdir, observable, allCorr['shortname'], centrange[0], centrange[1]))

In [ ]:
# make fully-subtracted correlation plots
for observable in observables:
    for centrange in centranges:
        fig = plt.figure(figsize=(len(photonptranges) * 8, 8))
        for (icol, photonptrange) in enumerate(photonptranges):
            plt.subplot(1, len(photonptranges), icol + 1)
            corr = allCorr[centrange][photonptrange][observable]
            corr.plotCorr('corr', fmt='o', label=allCorr['longname'])
                        
            if observableInfo[observable]['isLog']:
                plt.yscale('log')
                
            plt.xlabel(observableInfo[observable]['xlabel'])
            
            if icol == 0:
                plt.ylabel(observableInfo[observable]['ylabel'])
                infotext = '{0}, {1}-{2}%'.format('Pb-Pb', centrange[0], centrange[1])
                plt.annotate(infotext, **observableInfo[observable]['systemtextplacement'])
                
            infotext = '{0} < $p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1} GeV/$c$'.format(photonptrange[0], photonptrange[1])
            for cut in observableInfo[observable]['cuts']:
                infotext = infotext + '\n' + observableInfo[observable]['cuts'][cut]['text']
            plt.annotate(infotext, **observableInfo[observable]['pttextplacement'])
            
        fig.savefig('{0}/{1}/corr-measured-{2}-{3}.png'.format(plotdir, observable, centrange[0], centrange[1]))

In [ ]:
# plot signal, background, and sum together
for observable in observables:
    for centrange in centranges:
        fig = plt.figure(figsize=(len(photonptranges) * 8, 8))
        for (icol, photonptrange) in enumerate(photonptranges):
            plt.subplot(1, len(photonptranges), icol + 1)
            corr = allCorr[centrange][photonptrange][observable]
            corr.plotCorr('corr', fmt='o', label='Signal')
            corr.plotCorr('totalbkg', fmt='s', label='Total bkg')
            corr.plotCorr('sesr', fmt='k.', label='SESR')
                        
            if observableInfo[observable]['isLog']:
                plt.yscale('log')
                
            plt.xlabel(observableInfo[observable]['xlabel'])
            
            if icol == 0:
                plt.ylabel(observableInfo[observable]['ylabel'])
                infotext = '{0}, {1}-{2}%'.format('Pb-Pb', centrange[0], centrange[1])
                plt.annotate(infotext, **observableInfo[observable]['systemtextplacement'])
                
            if icol == len(photonptranges) - 1:
                plt.legend(loc=observableInfo[observable]['legendLoc'])
                
            infotext = '{0} < $p_\mathrm{{T}}^\mathrm{{cluster}}$ < {1} GeV/$c$'.format(photonptrange[0], photonptrange[1])
            for cut in observableInfo[observable]['cuts']:
                infotext = infotext + '\n' + observableInfo[observable]['cuts'][cut]['text']
            plt.annotate(infotext, **observableInfo[observable]['pttextplacement'])
            
        fig.savefig('{0}/{1}/corr-signal-bkg-{2}-{3}.png'.format(plotdir, observable, centrange[0], centrange[1]))

## Tables

In [ ]:
rootfile = ROOT.TFile.Open(rootfileSE)
sehTrigSR = rootfile.Get('hTrigSR')
rootfile.Close()

purityconfig = configall['purity']['functionparams']
purityptranges = getRanges(purityconfig['binEdges'])
puritycentranges = [tuple(cv['centralityrange']) for cv in purityconfig['centralityvalues']]
purityvalues = {tuple(cv['centralityrange']): {tuple(purityptranges[i]): cv['values'][i] for i in range(len(purityptranges))} for cv in purityconfig['centralityvalues']}

def getNsigNbkg(centrange, ptrange):
    nsig = 0
    nbkg = 0
    for crange in puritycentranges:
        if crange[0] >= centrange[1] or crange[1] <= centrange[0]:
            continue
        for prange in purityptranges:
            if prange[0] >= ptrange[1] or prange[1] <= ptrange[0]:
                continue
            
            slices = []
            slices.append((AxisNum.centrality.value, crange[0], crange[1]))
            slices.append((AxisNum.clusterpt.value, prange[0], prange[1]))
            ntrig = sliceAndProjectTHnSparse(sehTrigSR, slices, 0).Integral()
            
            purity = purityvalues[crange][prange]
            nsig = nsig + purity * ntrig
            nbkg = nbkg + (1 - purity) * ntrig
            
    return nsig, nbkg

print('Delta phi trigger/mix counts; relerr = sqrt(1+B/S)/sqrt(S)')
print('Signal and background calculated via purity and ntrig SESR')
print('-------------------------------------------------------------------------------------------')
print('Centrality| Cluster pT||ntrig SESR|ntrig SEBR|nmix MESR|nmix MEBR||Signal|Background|relerr')
for centrange in centranges:
    print('-------------------------------------------------------------------------------------------')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['deltaphi']
        sesrntrig = int(corr.sesrntrig)
        sebrntrig = int(corr.sebrntrig)
        srnmix = int(corr.srnmix)
        brnmix = int(corr.brnmix)
        nsig, nbkg = getNsigNbkg(centrange, photonptrange)
        precision = np.sqrt(1 + nbkg / nsig) / np.sqrt(nsig)
        print('{0:6d}-{1}%|{2}-{3} GeV/c||{4:10d}|{5:10d}|{6:9d}|{7:9d}||{8:6d}|{9:10d}|{10:6.4f}'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesrntrig, sebrntrig, srnmix, brnmix, int(round(nsig)), int(round(nbkg)), precision))


In [ ]:
print('Number of pairs in ptratio plot')
print('------------------------------------------------------------------')
print('Centrality|Cluster pT |npair SESR|npair SEBR|npair MESR|npair MEBR')
for centrange in centranges:
    print('------------------------------------------------------------------')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['ptratio']
        sesrnpair = int(corr.sesrth1.GetEntries())
        sebrnpair = int(corr.sebrth1.GetEntries())
        mesrnpair = int(corr.mesrth1.GetEntries())
        mebrnpair = int(corr.mebrth1.GetEntries())
        print('{0:6d}-{1}%|{2}-{3} GeV/c|{4:10d}|{5:10d}|{6:10d}|{7:10d}'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesrnpair, sebrnpair, mesrnpair, mebrnpair))


In [ ]:
print('Number of pairs in deltaphi plot')
print('------------------------------------------------------------------')
print('Centrality|Cluster pT |npair SESR|npair SEBR|npair MESR|npair MEBR')
for centrange in centranges:
    print('------------------------------------------------------------------')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['deltaphi']
        sesrnpair = int(corr.sesrth1.GetEntries())
        sebrnpair = int(corr.sebrth1.GetEntries())
        mesrnpair = int(corr.mesrth1.GetEntries())
        mebrnpair = int(corr.mebrth1.GetEntries())
        print('{0:6d}-{1}%|{2}-{3} GeV/c|{4:10d}|{5:10d}|{6:10d}|{7:10d}'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesrnpair, sebrnpair, mesrnpair, mebrnpair))


In [ ]:
print('Per-trigger yield for deltaphi > {0}pi/{1}'.format(observableInfo['deltaphi']['nbins'] - 1, observableInfo['deltaphi']['nbins']))
print('---------------------------------------------------------------------------')
print('Centrality|Cluster pT | SESR | SEBR | MESR | MEBR |Signal|Total bkg|Sig/bkg')
for centrange in centranges:
    print('---------------------------------------------------------------------------')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['deltaphi']
        sesryield = corr.sesrhist[-1]
        sebryield = corr.sebrhist[-1]
        mesryield = corr.mesrhist[-1]
        mebryield = corr.mebrhist[-1]
        sigyield = corr.corrhist[-1]
        bkgyield = corr.totalbkghist[-1]

        print('{0:6d}-{1}%|{2}-{3} GeV/c|{4:6.4f}|{5:6.4f}|{6:6.4f}|{7:6.4f}|{8:6.3f}|{9:9.4f}|{10:7.4f}'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesryield, sebryield, mesryield, mebryield, sigyield, bkgyield, sigyield / bkgyield))


In [ ]:
print('Per-trigger yield in 0 < ptratio < 1')
print('SESR, SEBR, MESR, MEBR are purity-weighted')
print('---------------------------------------------------------------------------')
print('Centrality|Cluster pT | SESR | SEBR | MESR | MEBR |Signal|Total bkg|Sig/bkg')
for centrange in centranges:
    print('---------------------------------------------------------------------------')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['ptratio']
        sesryield = sum(corr.sesrhist[0:4])
        sebryield = sum(corr.sebrhist[0:4])
        mesryield = sum(corr.mesrhist[0:4])
        mebryield = sum(corr.mebrhist[0:4])
        sigyield = sum(corr.corrhist[0:3])
        bkgyield = sum(corr.totalbkghist[0:4])

        print('{0:6d}-{1}%|{2}-{3} GeV/c|{4:6.3f}|{5:6.3f}|{6:6.3f}|{7:6.3f}|{8:6.3f}|{9:9.3f}|{10:7.4f}'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesryield, sebryield, mesryield, mebryield, sigyield, bkgyield, sigyield / bkgyield))


## LaTeX tables

In [ ]:
rootfile = ROOT.TFile.Open(rootfileSE)
sehTrigSR = rootfile.Get('hTrigSR')
rootfile.Close()

purityconfig = configall['purity']['functionparams']
purityptranges = getRanges(purityconfig['binEdges'])
puritycentranges = [tuple(cv['centralityrange']) for cv in purityconfig['centralityvalues']]
purityvalues = {tuple(cv['centralityrange']): {tuple(purityptranges[i]): cv['values'][i] for i in range(len(purityptranges))} for cv in purityconfig['centralityvalues']}

def getNsigNbkg(centrange, ptrange):
    nsig = 0
    nbkg = 0
    for crange in puritycentranges:
        if crange[0] >= centrange[1] or crange[1] <= centrange[0]:
            continue
        for prange in purityptranges:
            if prange[0] >= ptrange[1] or prange[1] <= ptrange[0]:
                continue
            
            slices = []
            slices.append((AxisNum.centrality.value, crange[0], crange[1]))
            slices.append((AxisNum.clusterpt.value, prange[0], prange[1]))
            ntrig = sliceAndProjectTHnSparse(sehTrigSR, slices, 0).Integral()
            
            purity = purityvalues[crange][prange]
            nsig = nsig + purity * ntrig
            nbkg = nbkg + (1 - purity) * ntrig
            
    return nsig, nbkg

print('Delta phi trigger/mix counts; relerr = sqrt(1+B/S)/sqrt(S)')
print('Signal and background calculated via purity and ntrig SESR')
print('')
print(r'\begin{tabular}{cc|cccc|ccc}')
print(r'\hline')
print(r'Centrality & Cluster $\pt$ & SESR & SEBR & MESR & MEBR & Sig & Bkg & $\sqrt{1+B/S}/\sqrt{S}$ \\')
print(r'\hline')
for centrange in centranges:
    print(r'\hline')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['deltaphi']
        sesrntrig = int(corr.sesrntrig)
        sebrntrig = int(corr.sebrntrig)
        srnmix = int(corr.srnmix)
        brnmix = int(corr.brnmix)
        nsig, nbkg = getNsigNbkg(centrange, photonptrange)
        precision = np.sqrt(1 + nbkg / nsig) / np.sqrt(nsig)
        print(r'{0}-{1}\% & {2}-{3} GeV/$c$ & {4} & {5} & {6} & {7} & {8} & {9} & {10:.4f} \\'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesrntrig, sebrntrig, srnmix, brnmix, int(round(nsig)), int(round(nbkg)), precision))
print(r'\hline')
print(r'\end{tabular}')

In [ ]:
print('Per-trigger yield for deltaphi > {0}pi/{1}'.format(observableInfo['deltaphi']['nbins'] - 1, observableInfo['deltaphi']['nbins']))
print('SESR, SEBR, MESR, MEBR are purity-weighted')
print('')
print(r'\begin{tabular}{cc|cccc|ccc}')
print(r'\hline')
print(r'Centrality & Cluster $\pt$ & SESR & SEBR & MESR & MEBR & Signal & Total bkg & Sig/bkg \\')
for centrange in centranges:
    print(r'\hline')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['deltaphi']
        sesryield = corr.sesrhist[-1]
        sebryield = corr.sebrhist[-1]
        mesryield = corr.mesrhist[-1]
        mebryield = corr.mebrhist[-1]
        sigyield = corr.corrhist[-1]
        bkgyield = corr.totalbkghist[-1]

        print(r'{0}-{1}\% & {2}-{3} GeV/$c$ & {4:.4f} & {5:.4f} & {6:.4f} & {7:.4f} & {8:.4f} & {9:.4f} & {10:.4f} \\'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesryield, sebryield, mesryield, mebryield, sigyield, bkgyield, sigyield / bkgyield))
print(r'\hline')
print(r'\end{tabular}')

In [ ]:
print('Per-trigger yield in 0 < ptratio < 1')
print('SESR, SEBR, MESR, MEBR are purity-weighted')
print('')
print(r'\begin{tabular}{cc|cccc|ccc}')
print(r'\hline')
print(r'Centrality & Cluster $\pt$ & SESR & SEBR & MESR & MEBR & Signal & Total bkg & Sig/bkg \\')
for centrange in centranges:
    print(r'\hline')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['ptratio']
        sesryield = sum(corr.sesrhist[0:4])
        sebryield = sum(corr.sebrhist[0:4])
        mesryield = sum(corr.mesrhist[0:4])
        mebryield = sum(corr.mebrhist[0:4])
        sigyield = sum(corr.corrhist[0:3])
        bkgyield = sum(corr.totalbkghist[0:4])

        print(r'{0}-{1}\% & {2}-{3} GeV/$c$ & {4:.4f} & {5:.4f} & {6:.4f} & {7:.4f} & {8:.4f} & {9:.4f} & {10:.4f} \\'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesryield, sebryield, mesryield, mebryield, sigyield, bkgyield, sigyield / bkgyield))
print(r'\hline')
print(r'\end{tabular}')

In [ ]:
print('Number of pairs in ptratio plot')
print('')
print(r'\begin{tabular}{cc|cccc}')
print(r'\hline')
print(r'Centrality & Cluster $\pt$ & SESR & SEBR & MESR & MEBR \\')
for centrange in centranges:
    print(r'\hline')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['ptratio']
        sesrnpair = int(corr.sesrth1.GetEntries())
        sebrnpair = int(corr.sebrth1.GetEntries())
        mesrnpair = int(corr.mesrth1.GetEntries())
        mebrnpair = int(corr.mebrth1.GetEntries())
        print(r'{0}-{1}\% & {2}-{3} GeV/$c$ & {4} & {5} & {6} & {7} \\'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesrnpair, sebrnpair, mesrnpair, mebrnpair))
print(r'\hline')
print(r'\end{tabular}')

In [ ]:
print('Number of pairs in deltaphi plot')
print('')
print(r'\begin{tabular}{cc|cccc}')
print(r'\hline')
print(r'Centrality & Cluster $\pt$ & SESR & SEBR & MESR & MEBR \\')
for centrange in centranges:
    print(r'\hline')
    for photonptrange in photonptranges:
        corr = allCorr[centrange][photonptrange]['deltaphi']
        sesrnpair = int(corr.sesrth1.GetEntries())
        sebrnpair = int(corr.sebrth1.GetEntries())
        mesrnpair = int(corr.mesrth1.GetEntries())
        mebrnpair = int(corr.mebrth1.GetEntries())
        print(r'{0}-{1}\% & {2}-{3} GeV/$c$ & {4} & {5} & {6} & {7} \\'.format(centrange[0], centrange[1], photonptrange[0], photonptrange[1], sesrnpair, sebrnpair, mesrnpair, mebrnpair))
print(r'\hline')
print(r'\end{tabular}')